In [1]:
from scipy.misc import *
import numpy as np
import pylab as plb
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from scipy.integrate import quad
from scipy.integrate import nquad
from scipy.misc import derivative
from ipywidgets import interact, interactive, fixed
from ipywidgets import widgets
from matplotlib.widgets import Slider, Button, RadioButtons, CheckButtons, TextBox # Matplotlib widgets
import pandas as pd
import emcee
import corner
from astropy import table as Table # For fast and easy reading / writing with tables using numpy library
from galpy.potential import MiyamotoNagaiPotential, NFWPotential, RazorThinExponentialDiskPotential, BurkertPotential # GALPY potentials

In [2]:
# In[ ]:

tt=Table.Table.read('coordinates.txt', format='ascii.tab') # import deflected images data

theta1=tt['theta1'] 
theta2=tt['theta2']
sigma=tt['sigma']

theta=np.zeros(len(theta1),float)
for i in range(len(theta1)):
    theta[i]=np.sqrt(theta1[i]**2+theta2[i]**2)

tt=Table.Table.read('alpha.txt', format='ascii.tab') #import circumference angles data
alpha=tt['alpha']

tt=Table.Table.read('Cosmological_distances.txt', format='ascii.tab')#import cosmolofical distances
D_ds=tt['D_ds'][0] 
D_d=tt['D_d'][0]
D_s=tt['D_s'][0]
SIGMA_CRIT=tt['SIGMA_CRIT'][0]


FC = np.pi/(180*3600) #conversion factor between arcs and radians

#Exponential Disk potential
def alpha1_ExpDisk(Sigma_0,h_r):
    GRADPOT1 = np.zeros((len(theta1)), float)
    def POTDEF1(TheTa1,theta):
        TheTa = np.sqrt(TheTa1**2+theta2[l]**2)
        R = D_d*TheTa
        Sigma = Sigma_0*np.exp(-D_d*TheTa/h_r)
        kappa = Sigma/SIGMA_CRIT
        return (2/theta1[l])*TheTa1*kappa/SIGMA_CRIT**2
    for l in range(len(theta1)):
        GRADPOT1[l]= quad(POTDEF1, 0, theta1[l], limit=100, args=(theta[l]))[0]
    return GRADPOT1*SIGMA_CRIT**2

def alpha2_ExpDisk(Sigma_0,h_r):
    GRADPOT1 = np.zeros((len(theta1)), float)
    def POTDEF1(TheTa2,theta):
        TheTa = np.sqrt(TheTa2**2+theta1[l]**2)
        R = D_d*TheTa
        Sigma = Sigma_0*np.exp(-D_d*TheTa/h_r)
        kappa = Sigma/SIGMA_CRIT
        return (2/theta2[l])*TheTa2*kappa/SIGMA_CRIT**2
    for l in range(len(theta1)):
        GRADPOT1[l]= quad(POTDEF1, 0, theta2[l], limit=100, args=(theta[l]))[0]
    return GRADPOT1*SIGMA_CRIT**2


In [3]:
#Function of the deflected images
def THETA(R,H,K,SIGMA_0, H_R):
    THETA1 = np.zeros((len(theta1)), float)
    THETA2 = np.zeros((len(theta1)), float)
    r=R*FC
    h=H*FC
    k=K*FC
    Beta1 = r*np.cos(alpha)+h
    Beta2 = r*np.sin(alpha)+k
    GRADPOT1_DISK=alpha1_ExpDisk(SIGMA_0,H_R)
    GRADPOT2_DISK=alpha2_ExpDisk(SIGMA_0,H_R)
    for l in range(len(theta1)):
        THETA1[l] = Beta1[l]+GRADPOT1_DISK[l]
        THETA2[l] = Beta2[l]+GRADPOT2_DISK[l]
    #Graphics of source and images
    fig = plt.figure()
#    ax = fig.add_subplot(111)
    plt.rcParams['figure.figsize'] =(5,5)
    plb.plot(Beta1/FC, Beta2/FC, '--r')
    plb.plot(theta1/FC, theta2/FC, 'ob')
    plb.plot(THETA1/FC, THETA2/FC, 'og')
    plb.xlim(-2.5,2.5)
    plb.ylim(-2.5,2.5)
    plb.legend(['Source', 'Observational data', 'model values'], loc='upper right', fontsize=15)
#    ax.set_aspect('equal', adjustable='box')

In [4]:
#Interactive function
A=interactive(THETA,R=widgets.FloatSlider(min=0.01, max=0.2, step=0.01, value=0.1),
        H=widgets.FloatSlider(min=-0.2, max=0.2, step=0.01, value=0.0),
        K=widgets.FloatSlider(min=-0.2, max=0.2, step=0.01, value=0.0),
        H_R=widgets.FloatSlider(min=2, max=24, step=0.2, value=12),
        SIGMA_0 = widgets.FloatSlider(min=1e8, max=60e8, step=0.5e8, value=30e8));
display(A)
button = widgets.Button(description="Finish")
output = widgets.Output()
display(button, output)
def on_button_clicked(b):
    with output:
        r=A.kwargs['R']
        h=A.kwargs['H']
        k=A.kwargs['K']
        h_r=A.kwargs['H_R']
        Sigma_0=A.kwargs['SIGMA_0']
        compnts = ['r', 'h','k','h_r','Sigma_0']
        values = [r, h, k, h_r, Sigma_0]
        error = ['None', 'None', 'None', 'None', 'None']
        init_parameters = Table.Table([compnts,values,error], names=('variable', 'value', 'error'))
        init_parameters.write('init_guess_params.txt', format='ascii.tab', overwrite=True)
        print("--------------Done---------------")
        print(init_parameters)
button.on_click(on_button_clicked)

interactive(children=(FloatSlider(value=0.1, description='R', max=0.2, min=0.01, step=0.01), FloatSlider(value…

Button(description='Finish', style=ButtonStyle())

Output()